# 0. Meta

## 0.1. Packages

In [31]:
import pandas as pd
from datetime import datetime
import numpy as np
import os
import joblib
import optuna
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, cross_val_predict
import logging
import optuna

optuna.logging.set_verbosity(optuna.logging.INFO)

## 0.2. Functions

# 1. Data Import

In [5]:
X_test = pd.read_csv("../data/processed/X_test.csv")
y_test = pd.read_csv("../data/processed/y_test.csv")
X_train = pd.read_csv("../data/processed/X_train.csv")
y_train = pd.read_csv("../data/processed/y_train.csv")

# 2. Model Training and Hyperparameter Tuning

## 2.1. Linear Regression

Train linear regression model and measure computation time.

In [6]:
lr_model = LinearRegression() 
lr_start = datetime.now()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_stop = datetime.now()
lr_delta = lr_stop - lr_start

Due to the log-transformation applied to the target variable y_train during data preprocessing to ensure normality and linearity, it is necessary to reverse this transformation on the predictions before assessing the model using metrics such as RMSE and R-squared.

In [7]:
y_test_log = np.log1p(y_test.to_numpy())

lr_pred_df = pd.DataFrame({'pred': lr_pred.flatten(), 'y_test': y_test_log.flatten()})

lr_pred_df['pred'] = np.expm1(lr_pred_df['pred'])
lr_pred_df['y_test'] = np.expm1(lr_pred_df['y_test'])

c:\Sync\03_projects\data_science\projects\autoscout24-car-price-prediction\envs\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in expm1
  result = getattr(ufunc, method)(*inputs, **kwargs)


During the back-transformation, NaNs and Infs occurred for some rows. These entries are eliminated, and the indices of the removed rows are stored to identify the observations that led to these issues. Additionally, the total count and the relative proportion of dropped rows are computed.

In [8]:
index_before = lr_pred_df.index
rows_before = lr_pred_df.shape[0]

lr_pred_df = lr_pred_df.replace([np.inf, -np.inf], np.nan).dropna()

index_after = lr_pred_df.index
rows_after = lr_pred_df.shape[0]

removed_indices = index_before.difference(index_after)
removed_rows = rows_before - rows_after
percent_removed = (removed_rows / rows_before) * 100

removed_rows_X_test = X_test.iloc[removed_indices]
removed_rows_y_test = y_test.iloc[removed_indices]
removed_rows_df = pd.concat([removed_rows_X_test, removed_rows_y_test], axis=1)
removed_rows_df['pre_backtrans_pred'] = lr_pred.flatten()[removed_indices]
filtered_removed_rows_df = removed_rows_df.loc[:, (removed_rows_df != 0).any()]

print('Rows before removing NaNs and Infs:', rows_before)
print('Rows after removing NaNs and Infs:', rows_after)
print('Number of rows removed:', removed_rows)
print('Percentage of rows removed:', percent_removed, '%')

print("Problematic rows in X_test and y_test:")
filtered_removed_rows_df

Rows before removing NaNs and Infs: 9215
Rows after removing NaNs and Infs: 9193
Number of rows removed: 22
Percentage of rows removed: 0.23874118285404233 %
Problematic rows in X_test and y_test:


,mileage,offerType,hp,year,make_Audi,make_BMW,make_Bentley,make_Corvette,make_DS,make_Fiat,...,model_S7,model_SLC 250,model_T5 Shuttle,fuel_Diesel,fuel_Electric,fuel_Gasoline,gear_Automatic,gear_Manual,price,pre_backtrans_pred
77,0.139248,0.50,0.538058,1.0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,83870,3.996071e+09
101,0.300000,0.00,0.732819,0.6,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,34950,5.998644e+09
824,0.406800,0.00,0.339380,0.5,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,8500,7.560187e+09
1032,0.283372,0.00,0.678830,0.7,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,35000,3.379601e+08
2314,0.403326,0.00,0.810958,0.0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,0,99800,3.948626e+10
2468,0.126833,0.00,0.770530,0.8,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,86885,3.379601e+08
2550,0.554074,0.00,0.493344,0.4,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,9985,5.998644e+09
3623,0.398366,0.00,0.744444,0.3,1,0,0,0,0,0,...,1,0,0,0,0,1,1,0,39925,7.201592e+09
3684,0.208001,0.50,0.538058,1.0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,45555,6.188076e+09
4405,0.020801,0.75,0.409779,1.0,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,33994,7.184330e+09


It appears that the linear regression model leads to extremely high predicted values for the problematic rows. Therefore, the back transformation fails, leading to infinite values. The full linear regression model therefore appears to be unsuitable. After removing the problematic rows, the evaluation metrics are calculated.  However, it should be noted that these can only be interpreted to a limited extent, as not all predictions are taken into account.

In [9]:
lr_r2 = r2_score(lr_pred_df['y_test'], lr_pred_df['pred'])
lr_r2_adj = 1 - (1 - lr_r2) * ((len(X_test) - 1) / (len(X_test) - len(X_test.columns) - 1))
lr_rmse = np.sqrt(mean_squared_error(lr_pred_df['y_test'], lr_pred_df['pred']))
lr_seconds = lr_delta.seconds + lr_delta.microseconds/1E6

lr_evaluation = pd.DataFrame({
    'model': ['lr'],
    'r2': [lr_r2],
    'r2_adj': [lr_r2_adj],
    'rmse': [lr_rmse],
    'seconds': [lr_seconds]
})

lr_evaluation

,model,r2,r2_adj,rmse,seconds
0,lr,0.928132,0.920122,4906.257588,15.906764


Save the trained model, model predictions and evaluation metrics.

In [10]:
joblib.dump(lr_model, '../models/models/lr_model.pkl')
lr_evaluation.to_csv(os.path.join("../models/evaluation/", "lr_evaluation.csv"), index=False)
lr_res = pd.DataFrame(lr_pred)
lr_res.index = X_test.index
lr_res.columns = ["prediction"]
lr_res.to_csv("../models/predictions/lr_prediction.csv")

## 2.2. Regularized Linear Regression

### 2.2.1. Lasso Regression

Start the hyperparameter optimization with an initial rough exploration of the hyperparameter space using Optuna's TPESampler algorithm (Tree-Structured Parzen Estimator), covering a wide range of values for the hyperparameter "alpha". This preliminary search helps to identify a suitable range for subsequent fine-tuning within the optimal parameter space. A 5-fold cross-validation strategy is used to optimize computational efficiency, which ensures robust model evaluation while reducing the risk of overfitting. The Root Mean Squared Error (RMSE) is used as a performance evaluation metric. 

In [13]:
def lasso_coarse_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-9, 1e9, log=True)
    lasso = Lasso(alpha=alpha)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(lasso, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [14]:
lasso_coarse_study = optuna.create_study(direction='minimize')
lasso_coarse_study.optimize(lasso_coarse_objective, n_trials=20)
lasso_coarse_best_params = lasso_coarse_study.best_params
lasso_coarse_best_params

[I 2024-02-09 19:04:18,003] A new study created in memory with name: no-name-3a79e641-7114-40c5-878d-6b257df9fb34
[I 2024-02-09 19:04:28,295] Trial 0 finished with value: 0.7013207160161288 and parameters: {'alpha': 10243321.458660385}. Best is trial 0 with value: 0.7013207160161288.
[I 2024-02-09 19:08:33,305] Trial 1 finished with value: 0.16394805195471082 and parameters: {'alpha': 5.96865180528638e-07}. Best is trial 1 with value: 0.16394805195471082.
[I 2024-02-09 19:08:38,564] Trial 2 finished with value: 0.7013207160161288 and parameters: {'alpha': 401158.3420221274}. Best is trial 1 with value: 0.16394805195471082.
[I 2024-02-09 19:08:43,155] Trial 3 finished with value: 0.7013207160161288 and parameters: {'alpha': 836.8044150727266}. Best is trial 1 with value: 0.16394805195471082.
[I 2024-02-09 19:08:47,370] Trial 4 finished with value: 0.7013207160161288 and parameters: {'alpha': 9.170761659151827}. Best is trial 1 with value: 0.16394805195471082.
[I 2024-02-09 19:08:51,996]

{'alpha': 1.9516900194665794e-06}

The highest test scores were achieved within the alpha range of 10^-6, therefore, proceed with hyperparameter optimization, emphasizing this optimal parameter space.

In [15]:
def lasso_fine_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-7, 1e-5, log=True)
    lasso = Lasso(alpha=alpha)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(lasso, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [16]:
lasso_fine_study = optuna.create_study(direction='minimize')
lasso_fine_study.optimize(lasso_fine_objective, n_trials=20)
lasso_fine_best_params = lasso_fine_study.best_params
lasso_fine_best_params

[I 2024-02-09 22:05:22,906] A new study created in memory with name: no-name-63e3e229-f876-40ed-8234-230ff6f3fbd4
[I 2024-02-09 22:09:08,328] Trial 0 finished with value: 0.16387247936991217 and parameters: {'alpha': 3.3649281346240794e-06}. Best is trial 0 with value: 0.16387247936991217.
[I 2024-02-09 22:12:05,381] Trial 1 finished with value: 0.1639890023075982 and parameters: {'alpha': 3.50098099799282e-07}. Best is trial 0 with value: 0.16387247936991217.
[I 2024-02-09 22:15:26,831] Trial 2 finished with value: 0.163960839806155 and parameters: {'alpha': 5.928976887895147e-06}. Best is trial 0 with value: 0.16387247936991217.
[I 2024-02-09 22:18:52,154] Trial 3 finished with value: 0.16397385111604876 and parameters: {'alpha': 6.14406310830036e-06}. Best is trial 0 with value: 0.16387247936991217.
[I 2024-02-09 22:22:16,639] Trial 4 finished with value: 0.1639914131708431 and parameters: {'alpha': 3.420306904942853e-07}. Best is trial 0 with value: 0.16387247936991217.
[I 2024-02-

{'alpha': 3.1599970196135527e-06}

Error is minimized at alpha = 3.1599970196135527e-06.

Train the model using the optimal hyperparameters, generate predictions for the test data using this trained model, evaluate the prediction performance, and display the results in a DataFrame. Additionally, save the fitted model as a pickle file, the model evaluation table, and the predictions as a CSV file.

In [17]:
lasso_model = Lasso(alpha=lasso_fine_best_params['alpha'])
lasso_start = datetime.now()
lasso_model.fit(X_train, y_train)
lasso_pred = lasso_model.predict(X_test)
lasso_stop = datetime.now()
lasso_delta = lasso_stop - lasso_start

c:\Sync\03_projects\data_science\projects\autoscout24-car-price-prediction\envs\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.949e+01, tolerance: 1.813e+00
  model = cd_fast.enet_coordinate_descent(


In [18]:
lasso_pred_inverse = np.expm1(lasso_pred)

lasso_r2 = r2_score(y_test, lasso_pred_inverse)
lasso_r2_adj = 1 - (1 - lasso_r2) * ((len(X_test) - 1) / (len(X_test) - len(X_test.columns) - 1))
lasso_rmse = np.sqrt(mean_squared_error(y_test, lasso_pred_inverse))
lasso_seconds = lasso_delta.seconds + lasso_delta.microseconds/1E6

lasso_evaluation = pd.DataFrame({
    'model': ['lasso'],
    'r2': [lasso_r2],
    'r2_adj': [lasso_r2_adj],
    'rmse': [lasso_rmse],
    'seconds': [lasso_seconds]
})

lasso_evaluation

,model,r2,r2_adj,rmse,seconds
0,lasso,0.949303,0.943652,4152.140941,53.182399


In [19]:
joblib.dump(lasso_model, '../models/models/lasso_model.pkl')
lasso_evaluation.to_csv(os.path.join("../models/evaluation/", "lasso_evaluation.csv"), index=False)
lasso_res = pd.DataFrame(lasso_pred)
lasso_res.index = X_test.index
lasso_res.columns = ["prediction"]
lasso_res.to_csv("../models/predictions/lasso_prediction.csv")

### 2.2.2. Ridge Regression

Start the ridge hyperparameter optimization with an initial rough exploration of the hyperparameter space using Optuna's TPESampler algorithm (Tree-Structured Parzen Estimator), covering a wide range of values for the hyperparameter "alpha". This preliminary search helps to identify a suitable range for subsequent fine-tuning within the optimal parameter space. A 5-fold cross-validation strategy is used to optimize computational efficiency, which ensures robust model evaluation while reducing the risk of overfitting. The Root Mean Squared Error (RMSE) is used as a performance evaluation metric. 

In [21]:
def ridge_coarse_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-9, 1e9, log=True)
    ridge = Ridge(alpha=alpha)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(ridge, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [22]:
ridge_coarse_study = optuna.create_study(direction='minimize')
ridge_coarse_study.optimize(ridge_coarse_objective, n_trials=20)
ridge_coarse_best_params = ridge_coarse_study.best_params
ridge_coarse_best_params

[I 2024-02-09 23:36:40,634] A new study created in memory with name: no-name-980e92ef-e28b-4d0c-9bce-ce0fa5a6cbc8
[I 2024-02-09 23:36:50,021] Trial 0 finished with value: 0.16424857262649997 and parameters: {'alpha': 1.1743120410150176}. Best is trial 0 with value: 0.16424857262649997.
[I 2024-02-09 23:36:55,919] Trial 1 finished with value: 0.17098125868126882 and parameters: {'alpha': 10.535200575024227}. Best is trial 0 with value: 0.16424857262649997.
[I 2024-02-09 23:37:01,370] Trial 2 finished with value: 0.18931491451252808 and parameters: {'alpha': 49.63657417215814}. Best is trial 0 with value: 0.16424857262649997.
[I 2024-02-09 23:37:07,791] Trial 3 finished with value: 0.6998189954419571 and parameters: {'alpha': 3193783.064577547}. Best is trial 0 with value: 0.16424857262649997.
[I 2024-02-09 23:37:15,744] Trial 4 finished with value: 0.418584998793132 and parameters: {'alpha': 3237.982475531299}. Best is trial 0 with value: 0.16424857262649997.
[I 2024-02-09 23:37:21,331]

{'alpha': 0.350978657363417}

The highest test scores were achieved within the alpha range of 10^-1 to 10^0, therefore, proceed with hyperparameter optimization, emphasizing this optimal parameter space.

In [23]:
def ridge_fine_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-1, 1e0, log=True)
    ridge = Ridge(alpha=alpha)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(ridge, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [24]:
ridge_fine_study = optuna.create_study(direction='minimize')
ridge_fine_study.optimize(ridge_fine_objective, n_trials=20)
ridge_fine_best_params = ridge_fine_study.best_params
ridge_fine_best_params

[I 2024-02-09 23:45:01,863] A new study created in memory with name: no-name-91b9b771-80b7-49b9-80a7-d1fbd13061b2
[I 2024-02-09 23:45:14,934] Trial 0 finished with value: 0.16400502975907394 and parameters: {'alpha': 0.7560482810597207}. Best is trial 0 with value: 0.16400502975907394.
[I 2024-02-09 23:45:22,776] Trial 1 finished with value: 0.16391336117257108 and parameters: {'alpha': 0.4982289691237173}. Best is trial 1 with value: 0.16391336117257108.
[I 2024-02-09 23:45:30,822] Trial 2 finished with value: 0.16389780865442807 and parameters: {'alpha': 0.38887340984488644}. Best is trial 2 with value: 0.16389780865442807.
[I 2024-02-09 23:45:36,991] Trial 3 finished with value: 0.16395869677401717 and parameters: {'alpha': 0.13233143481954387}. Best is trial 2 with value: 0.16389780865442807.
[I 2024-02-09 23:45:44,444] Trial 4 finished with value: 0.16405648400067147 and parameters: {'alpha': 0.8585941195554095}. Best is trial 2 with value: 0.16389780865442807.
[I 2024-02-09 23:45

{'alpha': 0.36415511361529757}

Error is minimized at alpha = 0.36415511361529757.

Train the ridge model using the optimal hyperparameters, generate predictions for the test data using this trained model, evaluate the prediction performance, and display the results in a DataFrame. Additionally, save the fitted model as a pickle file, the model evaluation table, and the predictions as a CSV file.

In [25]:
ridge_model = Ridge(alpha=ridge_fine_best_params['alpha'])
ridge_start = datetime.now()
ridge_model.fit(X_train, y_train)
ridge_pred = ridge_model.predict(X_test)
ridge_stop = datetime.now()
ridge_delta = ridge_stop - ridge_start

In [26]:
ridge_pred_inverse = np.expm1(ridge_pred)

ridge_r2 = r2_score(y_test, ridge_pred_inverse)
ridge_r2_adj = 1 - (1 - ridge_r2) * ((len(X_test) - 1) / (len(X_test) - len(X_test.columns) - 1))
ridge_rmse = np.sqrt(mean_squared_error(y_test, ridge_pred_inverse))
ridge_seconds = ridge_delta.seconds + ridge_delta.microseconds/1E6

ridge_evaluation = pd.DataFrame({
    'model': ['ridge'],
    'r2': [ridge_r2],
    'r2_adj': [ridge_r2_adj],
    'rmse': [ridge_rmse],
    'seconds': [ridge_seconds]
})

ridge_evaluation

,model,r2,r2_adj,rmse,seconds
0,ridge,0.940112,0.933437,4512.857727,1.733712


In [27]:
joblib.dump(ridge_model, '../models/models/ridge_model.pkl')
ridge_evaluation.to_csv(os.path.join("../models/evaluation/", "ridge_evaluation.csv"), index=False)
ridge_res = pd.DataFrame(ridge_pred)
ridge_res.index = X_test.index
ridge_res.columns = ["prediction"]
ridge_res.to_csv("../models/predictions/ridge_prediction.csv")

### 2.2.3. Elastic Net Regression

Start the elastic net hyperparameter optimization with an initial rough exploration of the hyperparameter space using Optuna's TPESampler algorithm (Tree-Structured Parzen Estimator), covering a wide range of values for the hyperparameters "alpha" and the "l1 ratio". This preliminary search helps to identify a suitable range for subsequent fine-tuning within the optimal parameter space. A 5-fold cross-validation strategy is used to optimize computational efficiency, which ensures robust model evaluation while reducing the risk of overfitting. The Root Mean Squared Error (RMSE) is used as a performance evaluation metric. 

In [32]:
def elasticnet_coarse_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-9, 1e9, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0, 1)
    elasticnet = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(elasticnet, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [33]:
elasticnet_coarse_study = optuna.create_study(direction='minimize')
elasticnet_coarse_study.optimize(elasticnet_coarse_objective, n_trials=20)
elasticnet_coarse_best_params = elasticnet_coarse_study.best_params
elasticnet_coarse_best_params

[I 2024-02-09 23:56:44,342] A new study created in memory with name: no-name-6bcc7919-3a04-4ee0-9f52-ca1229989486
[I 2024-02-09 23:56:51,473] Trial 0 finished with value: 0.7013207160161288 and parameters: {'alpha': 73.05249792172569, 'l1_ratio': 0.48929492242600703}. Best is trial 0 with value: 0.7013207160161288.
[I 2024-02-10 00:00:09,481] Trial 1 finished with value: 0.16428393249434645 and parameters: {'alpha': 3.698522250451831e-09, 'l1_ratio': 0.8038315597375395}. Best is trial 1 with value: 0.16428393249434645.
[I 2024-02-10 00:00:13,032] Trial 2 finished with value: 0.7013207160161288 and parameters: {'alpha': 1.5506251802243947, 'l1_ratio': 0.7408186330293501}. Best is trial 1 with value: 0.16428393249434645.
[I 2024-02-10 00:00:20,307] Trial 3 finished with value: 0.19518038543879007 and parameters: {'alpha': 0.0009063432259673031, 'l1_ratio': 0.3853263311447038}. Best is trial 1 with value: 0.16428393249434645.
[I 2024-02-10 00:03:35,356] Trial 4 finished with value: 0.1641

{'alpha': 2.430224145742283e-06, 'l1_ratio': 0.26693419316393463}

The highest test scores were achieved within the alpha range of 10^-6 and l1 ratio 10^-1 to 10^0, therefore, proceed with hyperparameter optimization, emphasizing this optimal parameter space.

In [34]:
def elasticnet_fine_objective(trial):
    alpha = trial.suggest_float('alpha', 1e-7, 1e-5, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0.1, 1)
    elasticnet = ElasticNet(alpha=alpha)
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = cross_val_score(elasticnet, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kfolds, n_jobs=-1)
    return -rmse_scores.mean()

In [35]:
elasticnet_fine_study = optuna.create_study(direction='minimize')
elasticnet_fine_study.optimize(elasticnet_fine_objective, n_trials=20)
elasticnet_fine_best_params = elasticnet_fine_study.best_params
elasticnet_fine_best_params

[I 2024-02-10 00:56:58,441] A new study created in memory with name: no-name-f942f13e-b614-4983-9dcf-80893e13c93e
[I 2024-02-10 01:00:24,000] Trial 0 finished with value: 0.16387115187660722 and parameters: {'alpha': 6.853742464288298e-06, 'l1_ratio': 0.9444272520565056}. Best is trial 0 with value: 0.16387115187660722.
[I 2024-02-10 01:03:32,391] Trial 1 finished with value: 0.1638834862773019 and parameters: {'alpha': 2.103684115749e-06, 'l1_ratio': 0.39341000050517383}. Best is trial 0 with value: 0.16387115187660722.
[I 2024-02-10 01:06:39,545] Trial 2 finished with value: 0.16406330363241867 and parameters: {'alpha': 3.0227541744550315e-07, 'l1_ratio': 0.4201850685184251}. Best is trial 0 with value: 0.16387115187660722.
[I 2024-02-10 01:09:40,722] Trial 3 finished with value: 0.1639364295174587 and parameters: {'alpha': 1.0178618975763246e-06, 'l1_ratio': 0.6600220041219447}. Best is trial 0 with value: 0.16387115187660722.
[I 2024-02-10 01:12:54,682] Trial 4 finished with value:

{'alpha': 5.0130395859195585e-06, 'l1_ratio': 0.848536496932069}

## 2.3. Gaussian Process Regression

## 2.4. Bayesian Linear Regression

## 2.5. Robust Regression

### 2.5.1. Huber Regression

### 2.5.2. Quantile Regression

### 2.5.3. RANSAC Regression

### 2.5.4. Theil Sen Regression

## 2.6. K-Nearest Neighbors Regression

## 2.7. Artificial Neural Networks

### 2.7.1. Multi-Layer Perceptron Regressor

## 2.8. Support Vector Regression

## 2.9. Decision Trees Regression

## 2.10. Ensemble

### 2.10.1. Ada Boost Regressor

### 2.10.2. Bagging Regressor

### 2.10.3. Extra Tree Regressor

### 2.10.4. Gradient Boosting Regressor

### 2.10.5. XGBoost Regressor

### 2.10.6. LightGBM Regressor

### 2.10.7. Random Forest Regressor

### 2.10.8. Extra Trees Regressor

### 2.10.9. Stacking Regressor

### 2.10.10. Voting Regressor

### 2.10.11. Histogram-based Gradient Boosting Regressor

## 2.10. Dimensionality-Reduced Regression